In [91]:
import torch
from h_net_dynamic_chunking import DynamicSequenceChunker

In [92]:
batch, seq, dim = 4, 10, 6
tokens = torch.randn(batch, seq, dim)

### Pack Chunker

In [93]:
# following section 2.2 of the paper

from collections import namedtuple

import torch
from torch import cat, arange
from torch.nested import nested_tensor
from torch.nn import Module, Linear, Parameter
from torch.nn.functional import cosine_similarity, pad
from torch.nn.utils.rnn import pad_sequence

from einx import multiply
from einops import repeat, rearrange

from assoc_scan import AssocScan

# constants

Outputs = namedtuple('Outputs', [
    'downsampled',
    'upsample_fn',
    'weighted_aux_ratio_loss'
])

Intermediates = namedtuple('Intermediates', [
    'mask',
    'probs',
    'chunk_lens',
    'boundary_mask',
    'residual',
    'gates',
    'upsampler_output_scale',
    'aux_ratio_loss',
    'input_mask',     # Added: needed to handle padding logic in upsampler
    'is_packed',      # Added: to track state
    'seq_lens',       # Added: original sequence lengths
    'new_seq_lens'    # Added: downsampled sequence lengths
])

# helper functions

def exists(v):
    return v is not None

def default(v, d):
    return v if exists(v) else d

def straight_through(t, value):
    return t + (value - t).detach()

def frac_gradient(t, frac = 1.):
    if frac == 1:
        return

    t_grad = t * frac
    return straight_through(t_grad, t)

# classes

class PackDynamicSequenceChunker(Module):
    def __init__(
        self,
        dim,
        dim_queries_keys = None,
        boundary_threshold = 0.5,
        target_avg_token_length = 6.,       # N in eq(10)
        ratio_loss_weight = 3e-2,
        handle_residual_proj = False,       # turning this on will automatically handle a projection of the residual and its application in the inverse upsample function
        assoc_scan_use_accelerated = False,
        learning_rate_difference = 0.75,    # in the paper, they report that as one moves up a hierarchy, the learning rate needs to decrease. we'll default to 0.75 for the rough 2.0 -> 1.5 somewhere in the appendix from level 0 -> 1
        straight_through_frac_vecs = True,  # improvisation where F receives gradients through straight-through with sigmoid
    ):
        super().__init__()
        dim_queries_keys = default(dim_queries_keys, dim)

        # linear to queries and keys

        self.to_queries_keys = Linear(dim, dim_queries_keys * 2, bias = False)

        # start key token, so first token can be segmented / chunked out

        self.start_key_token = Parameter(torch.randn(dim_queries_keys) * 1e-2) # presumably, need a start key token for the first token, open an issue if i got it wrong

        # threshold to determine boundary

        assert 0. < boundary_threshold < 1.

        self.boundary_threshold = boundary_threshold

        # smoothing related

        self.smooth_assoc_scan = AssocScan(use_accelerated = assoc_scan_use_accelerated)

        # maybe residual proj

        self.handle_residual_proj = handle_residual_proj

        if handle_residual_proj:
            self.residual_proj = Linear(dim, dim)

        # learning rate modulation, appendix C
        # the multiplier on the learning rate as one goes from outer to inner of the h-net, and inverse of this value from inner to outer

        self.learning_rate_difference = learning_rate_difference

        # ratio aux loss related

        self.target_avg_token_length = target_avg_token_length

        self.straight_through_frac_vecs = straight_through_frac_vecs

        self.ratio_loss_weight = ratio_loss_weight

        self.register_buffer('zero', torch.tensor(0.), persistent = False)

    def upsample(
        self,
        downsampled,
        intermediates: Intermediates,
        apply_scale = True
    ):
        # handle packed sequence input for upsampling
        is_packed = downsampled.ndim == 2
        
        if is_packed:
            # If the original input was packed, the downsampled input here should be packed.
            # We need to unpack it to (B, N, D) to use the internal logic.
            assert exists(intermediates.new_seq_lens), "Cannot upsample packed sequence without new_seq_lens"
            downsampled_list = list(downsampled.split(intermediates.new_seq_lens.tolist()))
            downsampled = pad_sequence(downsampled_list, batch_first=True)

        batch, needs_grad, device = downsampled.shape[0], downsampled.requires_grad, downsampled.device

        mask = intermediates.mask
        gates = intermediates.gates
        residual = intermediates.residual

        # smoothing module for improved gradients eq(5)

        downsampled = self.smooth_assoc_scan(gates, downsampled)

        # upsample

        downsampled_without_padding = downsampled[mask]
        chunk_lens_without_padding = intermediates.chunk_lens[mask]

        seq = arange(downsampled_without_padding.shape[0], device = device)

        repeated_indices = torch.repeat_interleave(seq, chunk_lens_without_padding, dim = 0)
        upsampled = downsampled_without_padding[repeated_indices]

        upsampled = rearrange(upsampled, '(b n) d -> b n d', b = batch)

        scale = intermediates.upsampler_output_scale

        if needs_grad and apply_scale and exists(scale):
            upsampled = multiply('b n d, b n', upsampled, scale)

        if self.handle_residual_proj:
            # We need to use the original residual (input tokens)
            # If we are in packed mode, the residual stored in intermediates is likely padded.
            # We apply the projection on the padded residual and mask later if needed.
            upsampled = upsampled + self.residual_proj(residual)

        upsampled = frac_gradient(upsampled, self.learning_rate_difference)
        
        # If the original input was packed, we must return a packed sequence.
        # We use the original sequence lengths from intermediates to repack.
        if intermediates.is_packed:
            # Mask out padding from the padded upsampled result
            # intermediates.input_mask contains the valid locations of the original sequence
            if exists(intermediates.input_mask):
                upsampled = upsampled[intermediates.input_mask]
            else:
                # Fallback if mask missing (shouldn't happen in packed flow)
                upsampled_list = [upsampled[i, :l] for i, l in enumerate(intermediates.seq_lens)]
                upsampled = cat(upsampled_list, dim=0)

        return upsampled

    def forward(
        self,
        tokens, # float[b n d] OR float[total_n d] if packed
        seq_lens = None, # Required if tokens is packed
        return_intermediates = False,
        return_only_chunk_lens = False
    ):
        # ----------------------------------------------------------------------
        # Handle Packed Sequences
        # ----------------------------------------------------------------------
        is_packed = tokens.ndim == 2
        input_mask = None
        
        if is_packed:
            assert exists(seq_lens), 'seq_lens must be provided for packed sequences'
            # Unpack to (Batch, MaxLen, Dim) for processing
            tokens_list = list(tokens.split(seq_lens.tolist()))
            tokens_padded = pad_sequence(tokens_list, batch_first=True)
            
            # Create mask for padding
            # This is critical for cosine_sim and loss masking
            max_len = tokens_padded.shape[1]
            batch_size = len(seq_lens)
            range_tensor = arange(max_len, device=tokens.device).expand(batch_size, max_len)
            input_mask = range_tensor < seq_lens.unsqueeze(1)
            
            # Switch tokens pointer to padded version
            tokens = tokens_padded
        else:
            # Standard batched input
            # Create a default mask of all True if not provided
            batch, length = tokens.shape[:2]
            input_mask = torch.ones((batch, length), device=tokens.device, dtype=torch.bool)
            seq_lens = torch.full((batch,), length, device=tokens.device, dtype=torch.long)

        batch, length, device = *tokens.shape[:2], tokens.device

        residual = tokens

        queries, keys = self.to_queries_keys(tokens).chunk(2, dim = -1)

        start_keys = repeat(self.start_key_token, 'd -> b 1 d', b = batch)

        keys = cat((start_keys, keys), dim = 1)

        # each query looks at the previous key
        # Handle padding for cosine similarity: avoid NaN by adding epsilon to norms or masking
        
        keys_shifted = keys[:, :-1]
        
        # Safe cosine similarity
        eps = 1e-6
        queries_norm = queries.norm(dim=-1, keepdim=True)
        keys_norm = keys_shifted.norm(dim=-1, keepdim=True)
        numerator = (queries * keys_shifted).sum(dim=-1)
        denominator = (queries_norm * keys_norm).clamp(min=eps).squeeze(-1)
        cosine_sim = numerator / denominator

        # Mask out cosine sim on padding to avoid garbage boundaries
        cosine_sim = cosine_sim.masked_fill(~input_mask, 1.0) # 1.0 sim -> 0.0 prob -> no boundary

        probs = (1. - cosine_sim) * 0.5 # cosine sim is -1. to 1., this transforms it to 0. to 1.

        boundary_mask = probs > self.boundary_threshold # bool[b n]

        boundary_mask[:, 0] = True # first token must always be boundary
        
        # Ensure padding tokens are never boundaries
        boundary_mask = boundary_mask & input_mask

        # compute some lengths, per chunk and number of chunks per batch

        num_chunks = boundary_mask.long().sum(dim = -1)

        boundary_mask_with_end = pad(boundary_mask, (0, 1), value = True)
        sel_indices = repeat(arange(boundary_mask_with_end.shape[-1], device = device), 'n -> b n', b = batch)[boundary_mask_with_end]

        sel_indices = nested_tensor(sel_indices.split((num_chunks + 1).tolist()), layout = torch.jagged, device = device)

        sel_indices = sel_indices.to_padded_tensor(padding = -1)

        mask = (sel_indices != -1)[:, 1:]

        chunk_lens = sel_indices[:, 1:] - sel_indices[:, :-1]
        chunk_lens.masked_fill_(~mask, 0)

        # early return chunk lens if using a trained module as a tokenizer

        if return_only_chunk_lens:
            if is_packed:
                # Return packed chunk lenses
                return chunk_lens[mask]
            return chunk_lens

        # downsampling - they show in their experiments that picking out the boundary tokens works just fine

        boundary_tokens = tokens[boundary_mask] # pick out boundary tokens

        tokens_nt = nested_tensor(boundary_tokens.split(num_chunks.tolist()), layout = torch.jagged, device = device, requires_grad = True)

        downsampled_tokens = tokens_nt.to_padded_tensor(padding = 0.)

        # smoothing module for improved gradients eq(5)

        probs_nt = nested_tensor(probs[boundary_mask].split(num_chunks.tolist()), layout = torch.jagged, device = device, requires_grad = True)

        boundary_probs = probs_nt.to_padded_tensor(padding = 0.)

        gates = 1. - boundary_probs

        downsampled_tokens = multiply('b n d, b n', downsampled_tokens, boundary_probs)

        # for the upsampler

        confidence = torch.where(boundary_mask, probs, 1. - probs)

        # defaults if not training

        upsampler_output_scale = None
        aux_loss = self.zero
        weighted_aux_loss = self.zero

        needs_grad = tokens.requires_grad

        if needs_grad:
            # straight through for 1. multiplier on the expanded processed boundary tokens
            
            # For scale, we only care about valid tokens
            if is_packed:
                upsampler_output_scale = straight_through(confidence, 1.) # Padded shape
                # We don't mask here because multiply() in upsampler handles shape, 
                # but valid gradients only come from valid indices.
            else:
                upsampler_output_scale = straight_through(confidence, 1.)

            # auxiliary ratio loss in section 2.3.2, eq (10)

            N = self.target_avg_token_length

            F = boundary_mask.float()
            
            # Mask the probs for G calculation so padding doesn't affect mean
            probs_masked = probs * input_mask.float()
            G = probs_masked.sum(dim = -1) / input_mask.float().sum(dim = -1).clamp(min=1.)

            # allow for a soft F to straight through - https://arxiv.org/abs/2505.22074

            if self.straight_through_frac_vecs:
                F_soft = (probs - self.boundary_threshold).sigmoid()
                F = straight_through(F_soft, F)

            # Mask F for mean calculation
            F = (F * input_mask.float()).sum(dim = -1) / input_mask.float().sum(dim = -1).clamp(min=1.)

            aux_ratio_loss = N / (N - 1) * ((N - 1) * F * G + (1. - F) * (1. - G))

            aux_loss = aux_ratio_loss.mean()
            weighted_aux_loss = aux_loss * self.ratio_loss_weight

        # intermediates
        
        # Calculate new seq lengths for packed return
        new_seq_lens = num_chunks
        
        intermediates = Intermediates(
            mask, probs, chunk_lens, boundary_mask, residual, gates, 
            upsampler_output_scale, aux_loss,
            input_mask, is_packed, seq_lens, new_seq_lens
        )

        # return the upsample function

        def upsample(downsampled, apply_scale = True):
            return self.upsample(downsampled, intermediates, apply_scale = apply_scale)

        # adjust learning rate

        downsampled_tokens = frac_gradient(downsampled_tokens, self.learning_rate_difference ** -1)

        # handle packed output
        
        if is_packed:
            # We computed downsampled_tokens as (B, N_chunks, D) padded.
            # We need to pack it back to (Total_Chunks, D)
            # We can use new_seq_lens (num_chunks) to create the mask for valid data
            valid_chunks_mask = arange(downsampled_tokens.shape[1], device=device).expand(batch, -1) < new_seq_lens.unsqueeze(1)
            downsampled_tokens = downsampled_tokens[valid_chunks_mask]

        # returning

        outputs = Outputs(downsampled_tokens, upsample, weighted_aux_loss)

        if not return_intermediates:
            return outputs

        return outputs, intermediates

### Testing

In [94]:
chunker = PackDynamicSequenceChunker(dim=dim)

In [95]:
output, intermediates = chunker(tokens, return_intermediates=True)

In [96]:
downsampled = output.downsampled
upsample_fn = output.upsample_fn

In [97]:
assert upsample_fn(downsampled).shape == tokens.shape
upsampled = upsample_fn(downsampled)

In [98]:
# we'll try packing the tokens into a 2d tensor
packed_tokens = tokens.view(-1, dim)
seq_lens = torch.full((batch,), seq, dtype=torch.long)

In [99]:
packed_output, packed_intermediates = chunker(packed_tokens, seq_lens=seq_lens, return_intermediates=True)

In [100]:
packed_downsampled = packed_output.downsampled
packed_upsample_fn = packed_output.upsample_fn

In [101]:
packed_upsampled = packed_upsample_fn(packed_downsampled)

In [102]:
packed_upsampled = packed_upsampled.view(batch, seq, dim)

In [103]:
torch.allclose(upsampled, packed_upsampled)

True

### More Testing

In [104]:
import random
tokens = []
for _ in range(batch):
    length = random.randint(5, 15)
    tokens.append(torch.randn(length, dim))

In [105]:
seq_lens = torch.tensor([t.shape[0] for t in tokens], dtype=torch.long)

In [106]:
sum(seq_lens)

tensor(42)

In [107]:
packed_tokens = cat(tokens, dim=0)

In [108]:
assert packed_tokens.shape == (sum(seq_lens), dim)

In [109]:
chunker = DynamicSequenceChunker(dim=dim)

In [110]:
ref_chunks = []
for t in tokens:
    out, intermediates = chunker(t.unsqueeze(0), return_intermediates=True)
    ref_chunks.append(out.downsampled.squeeze(0))

In [111]:
pack_chunker = PackDynamicSequenceChunker(dim=dim)
pack_chunker.load_state_dict(chunker.state_dict())

<All keys matched successfully>

In [112]:
packed_chunks, packed_intermediates = pack_chunker(packed_tokens, seq_lens=seq_lens, return_intermediates=True)

In [119]:
packed_intermediates.new_seq_lens

tensor([4, 6, 5, 5])

In [120]:
# if we reshape packed_chunks to a list of tensors, make it nested, pad with zeros, and compare to ref_chunks, it should be equal
split = packed_chunks.downsampled.split(packed_intermediates.new_seq_lens.tolist())

In [128]:
split

(tensor([[ 0.2176,  0.0837, -0.4048,  0.0278,  0.1762, -0.4847],
         [-0.2892,  0.5990,  0.4893,  0.1364,  0.2473,  0.3044],
         [ 0.8975,  0.5271, -1.0416, -0.9485,  0.2752, -0.7261],
         [-0.1160,  0.0555,  0.3612, -0.1121,  0.1605, -0.2061]],
        grad_fn=<SplitWithSizesBackward0>),
 tensor([[-0.1043, -0.6508, -0.0214,  1.0781,  0.1795, -0.0027],
         [-0.5085, -0.0200, -0.3196, -0.4988, -0.4018, -0.9098],
         [ 0.0463,  0.7482,  0.1681,  0.7465,  1.1942, -0.7251],
         [-0.6641,  0.8126, -0.3511,  0.2983,  0.1297, -0.3710],
         [ 0.5654,  0.2293, -0.5117, -0.9868, -0.2734, -0.3189],
         [ 0.8158,  0.0167,  0.1516, -0.3882,  0.1960, -0.0573]],
        grad_fn=<SplitWithSizesBackward0>),
 tensor([[-0.2785, -0.2164,  0.7948, -1.0790,  0.0179, -0.5494],
         [ 0.5015, -0.3383,  0.6701,  0.5496,  0.7364, -0.0068],
         [ 0.1464, -1.0260,  0.2193,  0.6515,  0.3094,  0.1059],
         [ 0.0068,  0.0257, -1.4260, -0.5139,  1.2761,  0.4803],


In [124]:
from torch.nested import nested_tensor, to_padded_tensor

In [125]:
ref_chunks_padded = to_padded_tensor(nested_tensor(ref_chunks), 0.0)

In [129]:
split_padded = to_padded_tensor(nested_tensor(list(split)), 0.0)

In [130]:
torch.allclose(ref_chunks_padded, split_padded)

True